# Shakespeare GPT -- GPT From Scratch!


In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Dataset Exploration and Modification


In [2]:
# read in input
with open('./input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
# inpeact the length and the first 1000 chars
print("Length of dataset in chars:", len(text))
print(text[:1000])

Length of dataset in chars: 1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for

In [4]:
# create an alphabet based on the input
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(chars)
print(vocab_size)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65


In [5]:
# time to tokenize the alphabet (character tokenizer)

# lets create a mapping for encoding (char to int) and decoding (int to char)
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s, i in stoi.items()}

encode = lambda s: [stoi[c] for c in s] # string to list of ints
decode = lambda l: ''.join([itos[i] for i in l]) # list of ints to string

print(encode("Hello World!"))
print(decode(encode("Hello World!")))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42, 2]
Hello World!


In [6]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

# Dataset Splitting


In [16]:
# splitting the dataset into a train (90%) set and a validation (10%) set
n = int(0.9 * len(data))
train = data[:n]
val = data[n:]

print("TRAIN:", train.shape)
print("VAL:", val.shape)

TRAIN: torch.Size([1003854])
VAL: torch.Size([111540])


In [56]:
# creating a batch with visualization

torch.manual_seed(314159365) # for reproducibility
batch_size = 4 # number of blocks in each batche
block_size = 8 # context length

def get_batch(split):
    # create a batch of inputs (x) and targets (y)
    
    data = train if split == 'train' else val # choose split to use
    
    idx = torch.randint(len(data) - block_size, (batch_size,)) # get index of random token
        
    # get inputs and targets based in index
    x = torch.stack([data[i : i + block_size] for i in idx])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in idx])
    
    return x, y

Xb, Yb = get_batch('train')
print("Inputs shape:", Xb.shape)
print("Inputs:")
print(Xb)
print("\nTargets shape:", Yb.shape)
print("Targets:\n")
print(Yb)

for b in range(batch_size):
    print(f'\nBlock #{b+1}:')
    for i in range(block_size):
        context = Xb[b, :i+1] # context 'slides' over input
        target = Yb[b, i]
        print(f'{context.tolist()} ===> {target}')

Inputs shape: torch.Size([4, 8])
Inputs:
tensor([[51, 47, 53,  6,  1, 58, 46, 53],
        [37, 30, 30, 17, 24, 10,  0, 21],
        [21,  5, 50, 50,  1, 54, 56, 53],
        [41, 43, 47, 60, 43,  0, 37, 53]])

Targets shape: torch.Size([4, 8])
Targets:

tensor([[47, 53,  6,  1, 58, 46, 53, 59],
        [30, 30, 17, 24, 10,  0, 21,  1],
        [ 5, 50, 50,  1, 54, 56, 53, 51],
        [43, 47, 60, 43,  0, 37, 53, 59]])

Block #1:
[51] ===> 47
[51, 47] ===> 53
[51, 47, 53] ===> 6
[51, 47, 53, 6] ===> 1
[51, 47, 53, 6, 1] ===> 58
[51, 47, 53, 6, 1, 58] ===> 46
[51, 47, 53, 6, 1, 58, 46] ===> 53
[51, 47, 53, 6, 1, 58, 46, 53] ===> 59

Block #2:
[37] ===> 30
[37, 30] ===> 30
[37, 30, 30] ===> 17
[37, 30, 30, 17] ===> 24
[37, 30, 30, 17, 24] ===> 10
[37, 30, 30, 17, 24, 10] ===> 0
[37, 30, 30, 17, 24, 10, 0] ===> 21
[37, 30, 30, 17, 24, 10, 0, 21] ===> 1

Block #3:
[21] ===> 5
[21, 5] ===> 50
[21, 5, 50] ===> 50
[21, 5, 50, 50] ===> 1
[21, 5, 50, 50, 1] ===> 54
[21, 5, 50, 50, 1, 54] ===> 

# Bigram Language Model


In [57]:
# lets start with a very simple language model: the Bigram
# we will use PyTorch modules rather than building it ourselves
# (see my makemore-CityNames repo for Bigram and Trigram models from scratch)

torch.manual_seed(314159265) # fro reproducibility

class Bigram(nn.Module):
    
    # initialize the model
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # create an 2D embedding table
    
    # forward pass
    def forward(self, idx, targets=None):
        
        # idx and targets are a both (B, T) tensors of integers where B = batch and T = time (or block)
        logits = self.token_embedding_table(idx) # (B, T, C) | C = channels (or vocab_size)
        
        if targets is None:
            loss = None
        else:
            # dimension manipulation
            B, T, C = logits.shape
            logits = logits.view((B*T, C))
            targets = targets.view((B*T))
            loss = F.cross_entropy(logits, targets) # get loss using cross_entropy
        return logits, loss
    
    # generate new tokens
    def generate(self, idx, max_new_tokens):
        # idx is a (B, T) tensor
        for _ in range(max_new_tokens):
            
            logits, loss = self(idx) # get predictions
            
            logits = logits[:, -1, :] # focus on last time step (B, C)
            probs = F.softmax(logits, dim=1) # get probabilities over rows (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # sample from probs distribution (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # concatenate new token (B, T+1)
        
        return idx
    
m = Bigram(vocab_size)
logits, loss = m(Xb, Yb)
print(logits.shape)
print(loss.item())
print("Expected loss:", -torch.log(torch.tensor([1/65])).item())

# lets generate tokens from the Bigram
idx = torch.zeros((1,1), dtype=torch.long) # get index (first token is index 0 or '/n')
out = m.generate(idx, max_new_tokens=100)[0].tolist() # generate the new tokens
print(decode(out)) # decode the tokens

torch.Size([32, 65])
4.797330856323242
Expected loss: 4.174387454986572

MtcZK!kHeruAiysl3sI
&
aalhb$GxSyyysk3RkdWF?Yk
&iqfF?oHRwm?cqAZxSb  eU$WkqZlD.gnN-zYcjoduqpR!NPZtqjjA


### Yay, garbage! But thats ok because the Bigram model is random right now... so let's train it!


In [58]:
# create a PyTorch optimizer (to see an optimizer (gradient descent) from scratch go to my makemore-CityNames repo)
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-03)

In [59]:
batch_size = 32
for steps in range(10000):
    Xb, Yb = get_batch('train') # get sample batch
    
    # evaluate loss
    logits, loss = m(Xb, Yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.434514045715332


In [60]:
# lets generate tokens from the trained Bigram
idx = torch.zeros((1,1), dtype=torch.long) # get index (first token is index 0 or '/n')
out = m.generate(idx, max_new_tokens=300)[0].tolist() # generate the new tokens
print(decode(out)) # decode the tokens


MARES:
S:

TENAwe, s grolasices mo tth;-dis, s ho:
I'dre ano, whoyrd cane ther covouave gower a hobicut!
ABurhe henghenoustej?
Thers m t3:

We

A'Dehad; be th'sthist s?
A:
TIONon:

O:
3d.
Amit;


a ano por pl mese y,
Fina; inoniak!

MENESA'annwole t the wisethfonon is hofe shoran sof;
A:
I:
woutheas


# The Math Behind Self-Attention !


In [61]:
# consider the following:

torch.manual_seed(314159265)

B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [62]:
# version 1: inefficient approach

# we want x[b, t] = mean(x[b,i]) where i <= t
x_bow = torch.zeros((B,T,C)) # bow: bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t, C)
        x_bow[b,t] = torch.mean(xprev, 0)


In [63]:
# version 2: efficient approach but not what we want yet

weights = torch.tril(torch.ones((T,T))) # get trianglular ones tensor
weights = weights / torch.sum(weights, 1, keepdims=True) # convert ones to distributions
mat = weights @ x  # matrix multiplication

torch.allclose(x_bow,mat) # evaluates to true if all values are the same

True

In [64]:
# version 3: efficient and uses Softmax
tril = torch.tril(torch.ones((T,T)))
weights = torch.zeros((T,T)) # get trianglular ones tensor
weights = weights.masked_fill(tril == 0, float('-inf'))
weights = F.softmax(weights, dim=1)
mat = weights @ x  # matrix multiplication

torch.allclose(x_bow,mat) # evaluates to true if all values are the same

True

### As we can see, we can use matrix multiplication to optimize the calculation of mean over the time steps


In [66]:
# version 4: Attention!
torch.manual_seed(314159265)

B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# a single Head of self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False) # token we are at
query = nn.Linear(C, head_size, bias=False) # past tokens we want context from
value = nn.Linear(C, head_size, bias=False) # communicates data if interested

k = key(x) # (B, T, head_size) | in cross attention, this would come from somewhere else
q = query(x) # (B, T, head_size)
weights = q @ k.transpose(-2, -1) # (B, T, head_size) @ (B, head_size, T) ===> (B, T, T) 

tril = torch.tril(torch.ones((T,T)))
weights = weights.masked_fill(tril == 0, float('-inf')) # delete for encoder attention blocks
weights = F.softmax(weights, dim=-1)

v = value(x) # in cross attention, this would come from somewhere else
out = weights @ v  # matrix multiplication

out.shape

torch.Size([4, 8, 16])

#### Notes:

- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- No notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode of tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other.
- _Self-Attention_ just means that the keys and values are produced from the same source as queries. In _Cross-Attention_, the queries get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- In an _encoder_ attention block, just delete the single line that does the masking with `tril`, allowing all tokens to communicate. The example above is a _decoder_ attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- _Scaled_ attention additionally divides `weights` by 1 / sqrt(head_size). This makes it so when Q, K are unit variance, `weights` will be unit variance too and Softmax will stay diffuse and not saturate too much i.e. softmax will start to act like one-hot encoding. Example below:


In [68]:
# Scaled attention
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
weights = q @ k.transpose(-2, -1) * head_size**-0.5 # scaled by head_size**-0.5
print(f'k var: {k.var().item():.4f} | q var: {q.var().item():.4f} | weights var: {weights.var().item():.4f}')

k var: 1.0471 | q var: 1.1018 | weights var: 1.1922
